In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [2]:
cities = pd.read_csv("output_data/cities.csv", encoding="utf-8")
cities.head()

,Unnamed: 0,city,cloud (%),country,date,humidity (%),lat,lng,max temp (F),wind (mph)
0,0,Rikitea,78,PF,1652046763,77,-23.1203,-134.9692,24.65,8.80
1,1,Castro,2,BR,1652046763,88,-24.7911,-50.0119,12.42,1.74
2,2,Vaini,20,TO,1652046763,69,-21.2000,-175.2000,26.09,4.12
3,3,Dhidhdhoo,94,MV,1652046764,70,6.8833,73.1000,29.38,7.96
4,4,Khatanga,45,RU,1652046764,96,71.9667,102.5000,-15.21,3.98


In [4]:
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Lng"]]

100

In [5]:
# Configure gmaps with API key:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

In [6]:
city_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Cloudiness"] == 0), :]
city_df = city_df.dropna(how='any')
city_df.reset_index(inplace=True)
del city_df['index']
city_df.head()

In [ ]:
hotellist = []

for i in range(len(city_df)):
    lat = city_df.loc[i]['Lat']
    lng = city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
city_df["Hotel Name"] = hotellist
city_df = city_df.dropna(how='any')
city_df.head()

In [ ]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in city_df.iterrows()]
locations = city_df[["Lat", "Lng"]]

In [ ]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig